In [14]:
import  torch
from transformers import CLIPProcessor, CLIPModel
import faiss
import pickle

In [15]:
# CLIP 모델 로드
device = torch.device("mps") # 해당 부분 쿠다로 변경하시면 됩니다~
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

/opt/anaconda3/envs/kmy_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [16]:
IMAGE_FAISS_PATH = "faiss_image.index"
IMAGE_PATHS_FILE = "image_paths.pkl"

In [17]:
def encode_text(text):
    inputs = processor(text=[text], return_tensors="pt").to(device)

    with torch.no_grad():
        text_features = clip_model.get_text_features(**inputs)

    return text_features.cpu().numpy().astype("float32")

def search_similar_images(text, top_k=5):
    global image_index, image_paths  
    text_vec = encode_text(text)
    distances, indices = image_index.search(text_vec, top_k)
    results = [image_paths[idx] for idx in indices[0]]

    return results, distances[0]  # 이미지 경로와 거리 반환


In [13]:
# FAISS 인덱스 및 이미지 경로 로드
image_index = faiss.read_index(IMAGE_FAISS_PATH)
with open(IMAGE_PATHS_FILE, "rb") as f:
    image_paths = pickle.load(f)

query_text = "a dog running in a park"

similar_images, distances = search_similar_images(query_text)

for i, (img, dist) in enumerate(zip(similar_images, distances)):
    print(f"Rank {i+1}: {img} (Distance: {dist:.4f})")


Rank 1: /root/.cache/kagglehub/datasets/abhinavnayak/catsvdogs-transformed/versions/1/train_transformed/dog846.jpg (Distance: 131.2417)
Rank 2: /root/.cache/kagglehub/datasets/abhinavnayak/catsvdogs-transformed/versions/1/train_transformed/dog510.jpg (Distance: 134.7405)
Rank 3: /root/.cache/kagglehub/datasets/abhinavnayak/catsvdogs-transformed/versions/1/train_transformed/dog493.jpg (Distance: 135.7166)
Rank 4: /root/.cache/kagglehub/datasets/abhinavnayak/catsvdogs-transformed/versions/1/train_transformed/dog135.jpg (Distance: 136.0946)
Rank 5: /root/.cache/kagglehub/datasets/abhinavnayak/catsvdogs-transformed/versions/1/train_transformed/dog552.jpg (Distance: 137.2330)
